In [1]:
import sys, gc, weakref

In [6]:
sys.getrefcount("dwlkdjhwkdhkqgkjdjhwkjddjdhwddwjhjdjwhdhjwjdhwjdqhwkjdhwkqjdh")

3

In [13]:
lst = [1, 2, 3]
print("init", sys.getrefcount(lst))

tp = (lst, 3, 4)
print("tuple", sys.getrefcount(lst))

lst2 = lst3 = lst
print("assign", sys.getrefcount(lst))

del tp, lst2, lst3
print("del", sys.getrefcount(lst))


def show_count(obj):
    print("show_count", sys.getrefcount(obj))

show_count(lst)
print("finish", sys.getrefcount(lst))

init 2
tuple 3
assign 5
del 2
show_count 4
finish 2


In [17]:
sys.getsizeof("")

49

In [19]:
sys.getsizeof("a"), sys.getsizeof("aa"), sys.getsizeof("питон")

(50, 51, 84)

In [22]:
sys.getsizeof({1: 1}), sys.getsizeof(set())

(232, 216)

In [24]:
sys.getsizeof([]), sys.getsizeof([1])

(56, 64)

In [41]:
class PointOrdinal:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.z = 3
        self.s = {i: i for i in range(100_000)}
        
        
class PointSlot:
    __slots__ = ("x", "y", "z", "s")
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.z = 3
        self.s = {i: i for i in range(100_000)}
        

po = PointOrdinal(1, 2)
ps = PointSlot(1, 2)

In [28]:
sys.getsizeof(po), sys.getsizeof(ps), sys.getsizeof(PointOrdinal), sys.getsizeof(PointSlot)

(48, 48, 1072, 904)

In [30]:
sys.getsizeof(po), sys.getsizeof(ps), sys.getsizeof(PointOrdinal), sys.getsizeof(PointSlot)

(48, 56, 1072, 904)

In [37]:
sys.getsizeof(po), sys.getsizeof(ps), sys.getsizeof(PointOrdinal), sys.getsizeof(PointSlot)

(48, 64, 1072, 904)

In [40]:
sys.getsizeof(po.s), sys.getsizeof(ps.s)

(5242968, 5242968)

In [42]:
sys.getsizeof(po.__dict__), sys.getsizeof(ps)

(104, 64)

In [45]:
po.xxx = 42
po.xxx1 = 42

In [47]:
sys.getsizeof(po.__dict__), sys.getsizeof(ps)

(144, 64)

In [48]:
import ctypes

In [54]:
lst = [1, 2, 3]
lst.append(lst)
print(lst)

sys.getrefcount(lst)

[1, 2, 3, [...]]


3

In [64]:
class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]


gc.disable()


lst = [1, 2, 3]
lst.append(lst)
print(lst)
print("ref init", sys.getrefcount(lst))

id_lst = id(lst)

del lst

# gc.collect()

print("final refcnt", PyObject.from_address(id_lst).refcnt)


[1, 2, 3, [...]]
ref init 3
final refcnt 1


In [71]:
len(gc.get_objects())

218828

In [66]:
del po, ps

In [82]:
class A:
    def __init__(self):
        self.x = 10
    
    def __del__(self):
        print("delete A")
        # super().__del__()

In [84]:
a = A()
b = c = a

print(sys.getrefcount(a))

4


In [85]:
del a

In [87]:
print(sys.getrefcount(b))


3


In [88]:
del b
print(sys.getrefcount(c))

2


In [89]:
del c

delete A


In [118]:
weakref.getweakrefcount?

In [120]:
a = A()
print("init", sys.getrefcount(a))

ref1 = weakref.ref(a)
ref2 = weakref.ref(a)
ref3 = weakref.ref(a)

print("weakref count", weakref.getweakrefcount(a))
print("weak init", sys.getrefcount(a))

a_ref = ref1()
print("weak call", sys.getrefcount(a))

print("is", a is a_ref)

del a_ref
print("weak del", sys.getrefcount(a))

print("weakref count", weakref.getweakrefcount(a))

print(weakref.getweakrefs(a))

del a

print("ref after del", ref())


init 2
weakref count 1
weak init 2
weak call 3
is True
weak del 2
weakref count 1
[<weakref at 0x107d344f0; to 'A' at 0x107019c60>]
delete A
ref after del None


In [103]:
a_ref = ref()

In [108]:
sys.getrefcount(a)

3

In [125]:
a = A()

def print_del():
    print("final del")

weakref.finalize(a, print_del)

b = c = a

print(sys.getrefcount(a))

4


In [122]:
del b, c

In [123]:
print(sys.getrefcount(a))

2


In [124]:
del a

delete A
final del


In [126]:
a = A()

def print_del():
    print("final del")

weakref.finalize(a, print_del)

b = c = a

print(sys.getrefcount(a))

delete A
final del
4


In [127]:
a.ref = a

In [128]:
del a, b, c

In [129]:
gc.collect()

final del
delete A
delete A


2420

In [130]:
import cProfile, pstats, io

In [131]:
def fib(n):
    if n < 2:
        return 1

    return fib(n - 1) + fib(n - 2)


pr = cProfile.Profile()
pr.enable()

fib(20)

pr.enable()


out = io.StringIO()
#sortby = 
ps = pstats.Stats(pr, stream=out)
ps.print_stats()

print(out.getvalue())

         22004 function calls (114 primitive calls) in 0.006 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 {method 'co_lines' of 'code' objects}
        4    0.000    0.000    0.000    0.000 {method 'replace' of 'code' objects}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        4    0.000    0.000    0.006    0.002 {built-in method builtins.exec}
        8    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
        5    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
       12    0.000    0.000    0.000    0.000 {built-in method builtins.next}
        4    0.000    0.000    0.000    0.000 /Library/Frameworks/Python.framework/Versions/3.10/